# On particle filters applied to electricity load forecasting

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib as mtp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import truncnorm
import scipy.stats as stats
from scipy.stats import invgamma
from scipy import ndimage
import math

### Parameters

In [2]:
M=10**3 #number of particles
n_pred=715  #number of predictions
N_daytype=8  #number of day types

### Load temperatures data 2015-2016

In [3]:
df_temp=pd.read_csv('data/temp_1516.csv',sep=',')
df_temp=df_temp.drop(df_temp.columns[0],axis=1)
df_temp=df_temp.sort_values(by='date')
df_temp.head()

,date,hour,t
0,2015-01-01,00:00,5.057414
1,2015-01-01,03:00,4.609138
2,2015-01-01,06:00,4.729828
3,2015-01-01,09:00,6.052241
4,2015-01-01,12:00,9.347069


In [4]:
#temperatures every day at 3pm
temp_day=df_temp[df_temp.hour=="03:00"]
temp_day.head()
#Array vector of temperatures for each day of the year
T_h=list(temp_day.t)

### Load electricity data

In [5]:
mypath="data/"
df=pd.read_csv(mypath+"cdc_conso_daytypes.csv",sep=",",encoding='latin-1')
df=df.drop(["daytype"+str(i) for i in range(0,9)],axis=1)
df=df.drop(df.columns[0],axis=1)
df.index=range(0,len(df))
df=df.sort_values(by=['Date'])
df['Date - Heure']=pd.to_datetime(df['Date - Heure'])
df.sort_values(by=["Date - Heure"])
#truncate to the data from 2015
df=df[(df.Date >='2015-01-01')]
df.head()

,Date - Heure,Date,Heure,Consommation (MW),Qualite donnee,daytype
166957,2015-01-01 03:30:00,2015-01-01,04:30,64838,DÃ©finitive,6
153294,2015-01-01 18:30:00,2015-01-01,19:30,72562,DÃ©finitive,6
153293,2015-01-01 18:00:00,2015-01-01,19:00,71766,DÃ©finitive,6
152017,2015-01-01 01:00:00,2015-01-01,02:00,71134,DÃ©finitive,6
152018,2015-01-01 04:30:00,2015-01-01,05:30,64379,DÃ©finitive,6


### Set variables of electricity demand

In [6]:
from datetime import date
df['Consommation']=df['Consommation (MW)']   #leave it in MW
#For prediction, every day at 3pm
consumption_day_ahead = list(df[df['Heure']=='15:00']['Consommation'])
#for initialization
consumption = list(df[df['Heure']=='15:00']['Consommation'][:30])

## Implementation of Algorithm 3.10

## At time n=0:

### 1.Definition of MCMC initial model at n=0 - sample $X_0^{j} \sim \mu(x_0)$

### Option A: Set values by hand

### Option C: MCMC using MHA or Gibbs (TBA)

#### Freeze parameters from Zak's simulation in gibbs-parameters_init_v1

In [7]:
N_daytype=9
k_day=npr.dirichlet(np.ones(N_daytype),1)
len_init=15
pred_forward=[0,1]

In [8]:
#Load pickle file
import pickle
output_file ='data/parameters_init_3am.pkl'
pkl_file = open(output_file, 'rb')
parameters_init = pickle.load(pkl_file)
parameters_init["x_init"][:20]

array([  3.02273798e+18,   6.67800469e+04,   6.46972701e+04,
         6.58113094e+04,   6.68636283e+04,   6.58854699e+04,
         6.59675720e+04,   6.58673126e+04,   6.58416274e+04,
         6.63315851e+04,   6.70433567e+04,   6.60284670e+04,
         6.66637126e+04,   6.56879587e+04,   6.70301774e+04,
         6.58260827e+04,   6.53871917e+04,   6.60484412e+04,
         6.52755648e+04,   6.55301609e+04])

In [9]:
parameters_init

{'g_heat_init': array([ -4.52154712e+17,  -3.80762497e+01,  -7.31525695e+03,
         -8.11152786e+03,  -8.58024628e+03,  -8.62612746e+03,
         -8.77134487e+03,  -8.85495467e+03,  -8.92782564e+03,
         -9.06289869e+03,  -9.22045359e+03,  -9.11176276e+03,
         -9.24379516e+03,  -9.13003974e+03,  -9.35915710e+03,
         -9.20423191e+03,  -9.16115433e+03,  -9.28044454e+03,
         -9.18334683e+03,  -9.23834433e+03,  -9.19391430e+03,
         -9.35277094e+03,  -9.41444952e+03,  -9.19184691e+03,
         -9.36655355e+03,  -9.42237059e+03,  -9.52888160e+03,
         -9.47623663e+03,  -9.27461548e+03,  -9.38874762e+03,
         -9.55896962e+03,  -9.36473862e+03,  -9.39112677e+03,
         -9.48519214e+03,  -9.51759661e+03,  -9.59170524e+03,
         -9.44740146e+03,  -9.38249020e+03,  -9.42309855e+03,
         -9.53570304e+03,  -9.50803386e+03,  -9.34696516e+03,
         -9.40641479e+03,  -9.62106348e+03,  -9.43971614e+03,
         -9.39556930e+03,  -9.56070872e+03,  -9.5365609

In [10]:
s_init=parameters_init['s_init']
g_heat_init=parameters_init['g_heat_init']
sigma_s_init=parameters_init['sigma_s_init']
sigma_g_init=parameters_init['sigma_g_init']
x_init=parameters_init['x_init']
sigma2=parameters_init['sigma2']
u_h=parameters_init['u_h']
kappa=parameters_init['kappa']
w_init=parameters_init['w_init']

In [11]:
sigma2=np.ones(M)*3*10**7
sigma_g_init=np.ones(M)*10**4
sigma_s_init=np.ones(M)*10**4
kappa=np.ones(M)

In [12]:
w_init[0]=np.median(w_init)
x_init[0]=np.median(x_init)
g_heat_init[0]=np.median(g_heat_init)
s_init[0]=np.median(s_init)

### Part 2: regularize weights and x if necessary

In [13]:
#STEP 2 OF PARTICLE FILTER
def resample(x_pred,w_prev,nbdays_pred_today,len_init,n,sigma_s,sigma_g,g_h,s,sigma,ESS):   
    #compute y_n
    delta_cons_gaus=-np.square(consumption_day_ahead[len_init+n+nbdays_pred_today]-x_pred)/(2*sigma**2)
    y_n=np.exp(delta_cons_gaus)
    #compute new weights
    if n>0:
        w_=w_prev*y_n
    else:
        w_=w_prev
    #likelihood of y_n
    lh_y_n=np.sum(delta_cons_gaus)
    #normalize weights
    w_h=w_/sum(w_)
    #calculate ESS
    ESS=1/sum(np.square(w_h))
    x =np.zeros(M)
    w =np.zeros(M)
    print("ESS of normalized weights=",round(ESS,6))
    if ESS <0.001*M: #reset the weights, keep x predicted as such
        print("ESS <0.001*M")
        x=x_pred
        if n==0:
            w=np.ones(M)*(1/M)
        w=w_prev
    elif (ESS>=0.001*M and ESS<0.5*M):  #reset all the weights, add some noise to a fraction of the x's
        print("ESS>=0.001*M and ESS_0<0.5*M")
        x,w,sigma_s,sigma_g,g_h,s=resample_multinomial(x_pred,w_h,sigma_s,sigma_g,g_h,s)
    elif ESS>=0.5*M:  #No degeneracy
        print("ESS>=0.5*M")
        x=x_pred
        w=w_h
    else:
        print("ESS critically low")
        x=x_pred
        if n==0:
            w=np.ones(M)*(1/M)
        w=w_prev
            
    print("new ESS=",round(1/sum(np.square(w)),6))
    return x,w,ESS,lh_y_n,sigma_s,sigma_g,g_h,s,w_h

In [14]:
def resample_multinomial(x_temp,w_temp,sigma_s,sigma_g,g_h,s):
    multinomial = np.random.multinomial(1,w_temp,M)
    new_x = np.zeros(M)
    new_s = np.zeros(M)
    new_g_heat = np.zeros(M)
    new_sigma_s = np.zeros(M)
    new_sigma_g = np.zeros(M)
    for i in range(M):
        new_x[i]=x_temp[np.argmax(multinomial[i,])]
        new_s[i]=s[np.argmax(multinomial[i,])]
        new_g_heat[i]=g_h[np.argmax(multinomial[i,])]
        new_sigma_s[i]=sigma_s[np.argmax(multinomial[i,])]
        new_sigma_g[i]=sigma_g[np.argmax(multinomial[i,])]
    new_w=(1/M)*np.ones(M)   
    return new_x,new_w,new_sigma_s,new_sigma_g,new_g_heat,new_s

In [15]:
#initialize matrix of x_heat, x_season
ESS=np.array(np.ones(n_pred+1))
x =np.zeros([n_pred+1,M])
w =np.zeros([n_pred+1,M])
lh_y_n =np.zeros(n_pred+1)
x_season =np.zeros([n_pred+1,M]) 
x_heat =np.zeros([n_pred+1,M])
entrop=np.zeros([len(pred_forward),n_pred])

x[0,:],w[0,:],ESS[0],lh_y,sigma_s_init,sigma_g_init,g_heat_init,s_init,entrop=resample(x_init,w_init,2,15,0,sigma_s_init,
                                                                                sigma_g_init,
                                                                                g_heat_init,s_init,sigma2**0.5,ESS)

ESS of normalized weights= 860.143299
ESS>=0.5*M
new ESS= 860.143299


## Prediction and Filtering at time n>0

### 1. Sample $x^j_n \mid X^j_{n-1} $ for all j=1...M particles

In [16]:
def x_season(day_type,k_day,s_prev,sigma_s_prev):
    nu=truncnorm.rvs(a = (-sigma_s_prev-0) / sigma_g_init , b = np.inf, loc= 0, scale = sigma_g_init, size=M)
    sigma_s=sigma_s_prev+nu
    err=truncnorm.rvs(a = -s_prev / sigma_s , b = np.inf, loc= 0, scale = sigma_s, size=M)
    s=s_prev+err
    x_s=s*k_day[day_type]
    return x_s, s, sigma_s

def x_heat(g_h_prev,day,sigma_g_prev):
    nu=truncnorm.rvs(a = -sigma_g_prev / sigma_g_init , b = np.inf, loc= 0, scale = sigma_g_init, size=M)
    sigma_g=sigma_g_prev+nu
    err=truncnorm.rvs(a = -np.inf , b =(-g_h_prev-0) / sigma_g, loc= 0, scale = sigma_g, size=M)
    g_h=g_h_prev+err
    if(u_h-T_h[day]<0):
        print("No heating effect")
    x_h=g_h*(T_h[day]-u_h)*max(np.sign(u_h-T_h[day]),0)
    return x_h, g_h, sigma_g

In [17]:
daytype=np.array(df.daytype)

In [18]:
#Initialize parameters
def particle_filter(nbdays_pred_today,len_init,len_filtering,s,g_h,sigma_s,sigma_g,sigma,lh_y):
    lh_y_n=np.zeros(len_filtering)
    x_pred=np.zeros([len_filtering,M])
    x_pred_mean=np.zeros(len_filtering)
    ESS=np.zeros(len_filtering)
    w_h=np.zeros([len_filtering,M])
    entrop_n=np.zeros([10,len_filtering])
    for n in range(1,len_filtering):
        print("n=",n)
        #prediction X[n] one day ahead, hourly forecast
        x_s=x_season(int(daytype[len_init+n+nbdays_pred_today]),kappa,s,sigma_s)
        x_h=x_heat(g_h,n+len_init+nbdays_pred_today,sigma_g)
        x_pred[n,:]=x_s[0]+x_h[0]
        x_pred_mean[n]=np.mean(x_pred[n,:])
        print("x_pred_mean =","{:.2e}".format(np.mean(x_pred[n,:])),
              "real consumption=","{:.2e}".format(consumption_day_ahead[n]))
        print("x_pred min=","{:.2e}".format(np.min(x_pred[n,:])),"x_pred max","{:.2e}".format(np.max(x_pred[n,:])))
        #take new values of parameters to feed x_season and x_heat in the next step
        s, sigma_s=x_s[1:]
        g_h, sigma_g=x_h[1:]
        #regularization
        x_pred[n,:],w[n,:],ESS[n],lh_y_n[n],sigma_s,sigma_g,g_h,s,w_h[n,:]=resample(x_pred[n,:],w[n-1,:],nbdays_pred_today,len_init,n,sigma_s,sigma_g,g_h,s,sigma,ESS)
        entrop_n[nbdays_pred_today,n-1]=-sum(w_h[n,:]*np.log(w_h[n,:]))
        print(entrop_n[nbdays_pred_today,n-1])
        print("likelihood:",lh_y_n[n])
        print("------------------------")
    return lh_y_n,x_pred_mean,ESS,entrop_n

In [19]:
fig=plt.figure(figsize=(12,6))
plt.plot(range(0,n_pred),entrop[0,:n_pred])
plt.title("Evolution of Entropy",fontweight='bold')
plt.show()

IndexError: too many indices for array

In [ ]:
fig=plt.figure(figsize=(12,6))
plt.plot(range(1,n_pred),ESS_calc[0,1:n_pred]/M)
#plt.plot(range(1,n_pred),ESS_calc[1,1:n_pred]/M)
#plt.plot(range(1,n_pred),ESS_calc[2,1:n_pred]/M)

plt.title("Evolution of ESS",fontweight='bold')
plt.xlabel('forecasted day')
plt.xlim(0,n_pred)
plt.show()

In [ ]:
fig=plt.figure(figsize=(12,6))


In [ ]:
#MAPE
mape_one_day_forecast=abs((x_predict[0,:]-consumption_day_ahead[:n_pred])/consumption_day_ahead[:n_pred])
mape_value = (1/(n_pred-pred_forward[0]))*sum(mape_one_day_forecast) 
print(mape_value)

In [ ]:
fig=plt.figure(figsize=(12,6))
plt.plot(range(n_pred),x_predict[0,:])
#plt.plot(range(n_pred),x_predict[1,:])
#plt.plot(range(n_pred),x_predict[2,:])
plt.plot(range(n_pred),consumption_day_ahead[:n_pred],linestyle='--')
plt.xlabel('forecasted day')
plt.ylim(10**4,10**5)
plt.title("Electricity load forecast in W",fontweight='bold')
plt.show()

Goal: re-estimate the one-dimensional parameters ($\sigma_g,\sigma_s,\mu_h,\kappa,\sigma$)<br>
Proposal distribution for these parameters: truncated gaussian random walk<br>
We set kappa constant at 1/8



## PMMH

#### Initialization of parameters

In [20]:
u_h_current=13
sigma_current=10**4
sigma_g_current=10**4
sigma_s_current=10**4

In [21]:
len_filter_pmmh=715
len_iter_mha=200
accept_log_proba=np.zeros(len_filter_pmmh)
accept_rate=0
log_lh_init=np.zeros(len_filter_pmmh)
lh_y_prop=np.zeros(len_filter_pmmh)

In [22]:
#store the accepted value in lists
u_h_list=[]
sigma_list=[]
sigma_g_list=[]
sigma_s_list=[]

#### Initialization of hyperparameters

In [23]:
#standard deviation of normal/trunc normal proposals on parameters
std_hyp_sigma_g,std_hyp_sigma_s,std_hyp_sigma=np.ones(3)*5*10**3
std_hyp_u=1

In [24]:
#joint prior density of parameters
def log_joint_prior(u_h,sigma,sigma_g,sigma_s):
    res=0
    res=(-(u_h-14)**2)/2
    res=res+(-0.01-1)*np.log(sigma**2) - (0.01/sigma**2)
    res=res+(-0.01-1)*np.log(sigma_g**2) - (0.01/sigma_g**2)
    res=res+(-0.01-1)*np.log(sigma_s**2) - (0.01/sigma_s**2)
    return res

In [25]:
#joint log prior density initialize
log_prior_init=log_joint_prior(u_h_current,sigma_current,sigma_g_current,sigma_s_current)
print(log_prior_init)
#initial parameters otbained from Gibbs. These initialized parameters will not change through iterations

-56.3146626545


#### Run initial particle filter and get the log likelihood

In [26]:
log_lh_init=particle_filter(0,len_init,
                            len_filter_pmmh,s_init,
                            g_heat_init,sigma_s_current,sigma_g_current,sigma_current,lh_y_n)[0]

n= 1
x_pred_mean = 8.24e+04 real consumption= 6.86e+04
x_pred min= 2.94e+01 x_pred max 7.39e+05
ESS of normalized weights= 319.91713
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -17804.5418653
------------------------
n= 2
x_pred_mean = 1.26e+05 real consumption= 6.41e+04
x_pred min= 1.88e+03 x_pred max 6.45e+05
ESS of normalized weights= 213.807395
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -54144.6725118
------------------------
n= 3
x_pred_mean = 1.07e+05 real consumption= 5.74e+04
x_pred min= 4.76e+03 x_pred max 9.87e+05
ESS of normalized weights= 299.39872
ESS>=0.001*M and ESS_0<0.5*M


C:\Users\Zakarya\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
C:\Users\Zakarya\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in multiply


new ESS= 1000.0
nan
likelihood: -36524.6074521
------------------------
n= 4
x_pred_mean = 1.26e+05 real consumption= 7.31e+04
x_pred min= 2.01e+03 x_pred max 7.19e+05
ESS of normalized weights= 269.292442
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -44023.4067297
------------------------
n= 5
x_pred_mean = 1.29e+05 real consumption= 7.68e+04
x_pred min= 1.52e+03 x_pred max 6.63e+05
ESS of normalized weights= 260.967481
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -49338.4193108
------------------------
n= 6
x_pred_mean = 1.19e+05 real consumption= 7.49e+04
x_pred min= 3.85e+03 x_pred max 6.43e+05
ESS of normalized weights= 293.103266
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -34476.135199
------------------------
n= 7
x_pred_mean = 1.23e+05 real consumption= 7.21e+04
x_pred min= 2.63e+03 x_pred max 6.98e+05
ESS of normalized weights= 298.543432
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -40708.8654091
--------------

new ESS= 1000.0
nan
likelihood: -26856.3995899
------------------------
n= 41
x_pred_mean = 8.00e+04 real consumption= 7.16e+04
x_pred min= 1.78e+03 x_pred max 3.96e+05
ESS of normalized weights= 438.194985
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
6.20784447676
likelihood: -7846.29654236
------------------------
n= 42
x_pred_mean = 1.06e+05 real consumption= 7.21e+04
x_pred min= 1.83e+03 x_pred max 3.89e+05
ESS of normalized weights= 314.162148
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
5.88409348926
likelihood: -20013.5442512
------------------------
n= 43
x_pred_mean = 1.35e+05 real consumption= 7.10e+04
x_pred min= 1.04e+04 x_pred max 6.78e+05
ESS of normalized weights= 193.445363
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -58778.3252072
------------------------
n= 44
x_pred_mean = 6.47e+04 real consumption= 6.39e+04
x_pred min= 3.04e+03 x_pred max 1.99e+05
ESS of normalized weights= 477.19179
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
6.28422329732
likelih

x_pred_mean = 1.16e+05 real consumption= 5.89e+04
x_pred min= 8.63e+03 x_pred max 5.96e+05
ESS of normalized weights= 297.203271
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -40878.0835486
------------------------
n= 77
x_pred_mean = 9.35e+04 real consumption= 6.31e+04
x_pred min= 4.78e+03 x_pred max 4.60e+05
ESS of normalized weights= 369.6829
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -18274.6672734
------------------------
n= 78
x_pred_mean = 8.18e+04 real consumption= 6.33e+04
x_pred min= 7.86e+03 x_pred max 3.09e+05
ESS of normalized weights= 410.019629
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
6.1312285161
likelihood: -11983.6864205
------------------------
n= 79
x_pred_mean = 1.18e+05 real consumption= 5.73e+04
x_pred min= 6.89e+03 x_pred max 7.22e+05
ESS of normalized weights= 185.51594
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -50344.161846
------------------------
n= 80
x_pred_mean = 1.29e+05 real consumption= 5.46e+04
x

...

new ESS= 1000.0
6.22058235138
likelihood: -3949.46215132
------------------------
n= 573
No heating effect
x_pred_mean = 4.68e+04 real consumption= 5.25e+04
x_pred min= 1.44e+03 x_pred max 2.03e+05
ESS of normalized weights= 701.958465
ESS>=0.5*M
new ESS= 701.958465
6.62947683536
likelihood: -1750.11552206
------------------------
n= 574
No heating effect
x_pred_mean = 4.83e+04 real consumption= 5.21e+04
x_pred min= 5.10e+02 x_pred max 1.90e+05
ESS of normalized weights= 445.809065
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
6.23917562705
likelihood: -3141.24654829
------------------------
n= 575
No heating effect
x_pred_mean = 4.72e+04 real consumption= 5.10e+04
x_pred min= 1.02e+03 x_pred max 1.77e+05
ESS of normalized weights= 675.81111
ESS>=0.5*M
new ESS= 675.81111
6.59361956897
likelihood: -2032.29708031
------------------------
n= 576
No heating effect
x_pred_mean = 4.97e+04 real consumption= 4.41e+04
x_pred min= 4.80e+01 x_pred max 2.75e+05
ESS of normalized weights= 427.402845

new ESS= 1000.0
6.19411041069
likelihood: -3970.83964621
------------------------
n= 611
No heating effect
x_pred_mean = 4.83e+04 real consumption= 4.58e+04
x_pred min= 9.81e+00 x_pred max 1.45e+05
ESS of normalized weights= 669.285291
ESS>=0.5*M
new ESS= 669.285291
6.59158624583
likelihood: -2007.69011988
------------------------
n= 612
No heating effect
x_pred_mean = 5.21e+04 real consumption= 4.14e+04
x_pred min= 2.92e+02 x_pred max 2.97e+05
ESS of normalized weights= 454.880049
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
6.26654700594
likelihood: -3631.40023069
------------------------
n= 613
No heating effect
x_pred_mean = 5.01e+04 real consumption= 5.29e+04
x_pred min= 2.33e+03 x_pred max 1.42e+05
ESS of normalized weights= 681.654281
ESS>=0.5*M
new ESS= 681.654281
6.59932415939
likelihood: -1843.94251256
------------------------
n= 614
No heating effect
x_pred_mean = 5.15e+04 real consumption= 5.43e+04
x_pred min= 7.72e+01 x_pred max 2.51e+05
ESS of normalized weights= 449.0382

new ESS= 1000.0
nan
likelihood: -35967.9294743
------------------------
n= 645
x_pred_mean = 9.68e+04 real consumption= 5.36e+04
x_pred min= 9.14e+03 x_pred max 4.77e+05
ESS of normalized weights= 307.170469
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -26564.9728521
------------------------
n= 646
x_pred_mean = 6.29e+04 real consumption= 4.66e+04
x_pred min= 4.26e+03 x_pred max 2.63e+05
ESS of normalized weights= 500.017346
ESS>=0.5*M
new ESS= 500.017346
6.32357643342
likelihood: -5471.22049754
------------------------
n= 647
No heating effect
x_pred_mean = 4.61e+04 real consumption= 4.32e+04
x_pred min= 5.78e+01 x_pred max 2.24e+05
ESS of normalized weights= 204.453768
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
5.55764802494
likelihood: -5818.27520088
------------------------
n= 648
No heating effect
x_pred_mean = 5.59e+04 real consumption= 5.58e+04
x_pred min= 9.65e+02 x_pred max 2.08e+05
ESS of normalized weights= 601.808872
ESS>=0.5*M
new ESS= 601.808872
6.502237

6.55044296098
likelihood: -2308.37953605
------------------------
n= 679
x_pred_mean = 1.01e+05 real consumption= 6.71e+04
x_pred min= 1.44e+04 x_pred max 3.81e+05
ESS of normalized weights= 189.62916
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
5.48565098753
likelihood: -15429.1623033
------------------------
n= 680
x_pred_mean = 1.34e+05 real consumption= 5.54e+04
x_pred min= 1.61e+04 x_pred max 5.56e+05
ESS of normalized weights= 151.382258
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -55864.5125849
------------------------
n= 681
x_pred_mean = 9.80e+04 real consumption= 5.98e+04
x_pred min= 6.41e+03 x_pred max 3.74e+05
ESS of normalized weights= 308.858395
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
5.85270361597
likelihood: -24833.1153187
------------------------
n= 682
x_pred_mean = 1.01e+05 real consumption= 5.49e+04
x_pred min= 4.04e+03 x_pred max 5.14e+05
ESS of normalized weights= 327.934607
ESS>=0.001*M and ESS_0<0.5*M
new ESS= 1000.0
nan
likelihood: -22654.

new ESS= 1000.0
nan
likelihood: -64731.4836491
------------------------


In [27]:
print(np.sum(log_lh_init))

-26119985.5891


#### PMMH Algorithm

In [1]:
for step in range(len_iter_mha):
    print("___________________________________________________________")
    print("Metropolis Hastings step:",step)
    #we need 6 inputs to compute the (log) acceptance probability log(r):
    #log_likelihood, joint prior density, log proposal density for both current parameters and proposed parameters
    #sample proposal for u_h, sigma, sigma_g, sigma_s
    u_h_prop=npr.normal(u_h_current,std_hyp_u,size=1)
    sigma_prop=stats.truncnorm.rvs(a=(0-sigma_current)/std_hyp_sigma,b=np.inf,loc=sigma_current,scale=std_hyp_sigma,size=1)
    sigma_g_prop=stats.truncnorm.rvs(a=(0-sigma_g_current)/std_hyp_sigma_g,b=np.inf,loc=sigma_g_current,scale=std_hyp_sigma_g,size=1)
    sigma_s_prop=stats.truncnorm.rvs(a=(0-sigma_s_current)/std_hyp_sigma_s,b=np.inf,loc=sigma_s_current,scale=std_hyp_sigma_s,size=1)
    print("proposed parameters:","u_heat:",u_h_prop,"sigma:",sigma_prop,"sigma_g:",sigma_g_prop,"sigma_s:",sigma_s_prop)
    #1/run a particle filter with the proposed parameters to obtain a an estimation of likelihood proposed
    #  consider the likelihood of the last day of the fitering
    lh_y_prop=particle_filter(0,len_init,len_filter_pmmh,s_init,
                              g_heat_init,sigma_s_prop,sigma_g_prop,sigma_prop,lh_y_n)[0]
    print("log likelihood proposal of y:",np.sum(lh_y_prop))
    #2/generate prior proposals and compute joint log density of them
    log_prior_prop = log_joint_prior(u_h_prop,sigma_prop,sigma_g_prop,sigma_s_prop)
    print("proposed log prior:",log_prior_prop)
    #3/compute log proposal density h(current parameter|proposed parameter)
    current_log_density=np.log(stats.norm.cdf(sigma_current/std_hyp_sigma,loc=0,scale=1))+np.log(
        stats.norm.cdf(sigma_s_current/std_hyp_sigma_s,loc=0,scale=1))+np.log(
        stats.norm.cdf(sigma_g_current/std_hyp_sigma_g,loc=0,scale=1))
    print("proposal log density initial parameters given proposed param:",current_log_density)
    #4/log likelihood from initial parameters --> already done: log_lh_init
    #5/joint prior of the initial parameters: we already have them
    #6/compute log proposal density h(proposed parameter|current parameter)
    prop_log_density=np.log(stats.norm.cdf(sigma_prop/std_hyp_sigma,loc=0,scale=1))+np.log(
    stats.norm.cdf(sigma_s_prop/std_hyp_sigma_s,loc=0,scale=1))+np.log(
    stats.norm.cdf(sigma_g_prop/std_hyp_sigma_g,loc=0,scale=1))
    print("proposal log density proposed parameters given current param:",prop_log_density)
    #we add up these elements to construct the log acceptance probability
    #numerator
    accept_log_proba[step]=np.sum(lh_y_prop)+log_prior_prop+current_log_density
    #denominator
    accept_log_proba[step]=accept_log_proba[step]-np.sum(log_lh_init)-log_prior_init-prop_log_density
    print("acceptance log probability:",accept_log_proba[step])
    u=npr.random()
    #to get an acceptance rate > 5%, we need log_proba to be at least -3
    if(np.log(u)<min(0,accept_log_proba[step])):
        print("ACCEPT")
        accept_rate=accept_rate+1
        log_lh_init=lh_y_prop
        sigma_current=sigma_prop
        sigma_g_current=sigma_g_prop
        sigma_s_current=sigma_s_prop
        u_h_current=u_h_prop
        #store the accepted values
        u_h_list.append(u_h_current)
        sigma_list.append(sigma_current)
        sigma_g_list.append(sigma_g_current)
        sigma_s_list.append(sigma_s_current)
    else:
        print("REJECT")

NameError: name 'len_iter_mha' is not defined

In [31]:
print(accept_rate/len_iter_mha)

0.29


In [ ]:
print(sigma_current)
print(sigma_g_current)
print(sigma_s_current)
print(u_h_current)

In [ ]:
u_h_list